In [ ]:
# STEP 1: Install dependencies
!pip install openpyxl statsmodels

# STEP 2: Import libraries
import pandas as pd
import numpy as np
import scipy.stats as stats
from statsmodels.stats.inter_rater import fleiss_kappa
from openpyxl.utils import column_index_from_string

# STEP 3: Load Excel file
file_path = "/content/sample_data/experts_clean_COMBINED.xlsx"
df = pd.read_excel(file_path, sheet_name="Sheet1")

# STEP 4: Define domain columns
domains = {
    "Food": ('Z', 'BD'),
    "LivingBeing": ('BG', 'CK'),
    "Tech": ('CN', 'DR'),
    "MythicalCreature": ('DU', 'EY'),
    "Instrument": ('FB', 'GF'),
    "Tool": ('GI', 'HM')
}

excluded_cols = ['BE', 'BF', 'CL', 'CM', 'DS', 'DT', 'EZ', 'FA', 'GG', 'GH', 'HN', 'HO', 'HP']
excluded_indices = [column_index_from_string(c) - 1 for c in excluded_cols]

# STEP 5: Optional confidence/struggle columns
confidence_cols = [col for col in df.columns if 'confidence' in col.lower()]
struggle_cols = [col for col in df.columns if 'struggle' in col.lower()]

# STEP 6: Fleiss' Kappa CI Function
def fleiss_kappa_ci(kappa_val, n_raters, n_items, alpha=0.05):
    se_kappa = np.sqrt((2 * (1 - kappa_val)) / (n_items * n_raters * (n_raters - 1)))
    z = kappa_val / se_kappa
    p_value = 2 * (1 - stats.norm.cdf(abs(z)))
    ci_half = stats.norm.ppf(1 - alpha / 2) * se_kappa
    return se_kappa, z, p_value, (max(-1.0, kappa_val - ci_half), min(1.0, kappa_val + ci_half))

# STEP 7: Function to process each domain
def process_domain(domain_name, start_col, end_col):
    print("\n" + "="*40)
    print(f"     Domain: {domain_name}")
    print("="*40)

    start_idx = column_index_from_string(start_col) - 1
    end_idx = column_index_from_string(end_col) - 1

    domain_cols = [
        df.columns[i] for i in range(start_idx, end_idx + 1)
        if i not in excluded_indices and df[df.columns[i]].dropna().isin([0, 1]).all()
    ]

    if not domain_cols:
        print(f"No valid binary columns found for domain {domain_name}. Skipping...")
        return

    # Agreement Rates
    agreement_rates = df[domain_cols].mean().sort_values(ascending=False)

    # Clean Fleiss matrix (drop incomplete rows)
    df_clean = df[domain_cols].dropna(axis=0, how='any')
    n_raters = df_clean.shape[0]

    fleiss_matrix = []
    for col in df_clean.columns:
        counts = df_clean[col].value_counts().reindex([0, 1], fill_value=0)
        fleiss_matrix.append([counts[0], counts[1]])
    fleiss_matrix = np.array(fleiss_matrix)

    # Fleiss' Kappa calculation
    kappa = fleiss_kappa(fleiss_matrix)
    se, z, p, ci = fleiss_kappa_ci(kappa, n_raters, len(domain_cols))
    ci_lower, ci_upper = float(ci[0]), float(ci[1])

    # Print Results
    print(f"✔️  Fleiss' Kappa Value     : {kappa:.4f}")
    print(f"📊  Z-Score                : {z:.4f}")
    p_str = f"{p:.2e} (p < 0.0001)" if p < 1e-4 else f"{p:.4f}"
    print(f"📈  p-Value (Z-test)       : {p_str}")
    print(f"🔒  95% Confidence Interval: ({ci_lower:.4f}, {ci_upper:.4f})")
    print(f"📌 Interpretation          : {'Statistically significant' if p < 0.05 else 'Not statistically significant'}")

    print("\n✅ Top 10 Agreement Rates:")
    print(agreement_rates.head(10))

    # Save agreement rates to CSV
    output_csv = f"agreement_rates_{domain_name}.csv"
    agreement_rates.to_csv(output_csv, float_format="%.4f", index_label="Concept", header=["AgreementRate"])
    print(f"\n💾 Agreement rates saved to: {output_csv}")

# STEP 8: Process each domain
for domain, (start_col, end_col) in domains.items():
    process_domain(domain, start_col, end_col)



     Domain: Food
✔️  Fleiss' Kappa Value     : 0.1154
📊  Z-Score                : 15.6945
📈  p-Value (Z-test)       : 0.00e+00 (p < 0.0001)
🔒  95% Confidence Interval: (0.1010, 0.1298)
📌 Interpretation          : Statistically significant

✅ Top 10 Agreement Rates:
apple           0.696970
grape           0.696970
broccoli        0.666667
cauliflower     0.636364
sweet potato    0.545455
carrot          0.545455
tuna filet      0.484848
steak           0.454545
filet           0.454545
pork sausage    0.424242
dtype: float64

💾 Agreement rates saved to: agreement_rates_Food.csv

     Domain: LivingBeing
✔️  Fleiss' Kappa Value     : 0.2244
📊  Z-Score                : 26.5813
📈  p-Value (Z-test)       : 0.00e+00 (p < 0.0001)
🔒  95% Confidence Interval: (0.2079, 0.2410)
📌 Interpretation          : Statistically significant

✅ Top 10 Agreement Rates:
Dog        0.814815
Cat        0.814815
Ostrich    0.740741
Penguin    0.740741
Eagle      0.740741
Owl        0.703704
Cedar      0.70370